# First importing libraries and changing the classification of probabilities

In [1]:
# needed libraries

import os, cv2
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
# Function from the main repo

def load_cell_image(fname):
    with Image.open(fname) as image:
        return np.asarray(image)
        
def load(fname='labels.csv'):
    data = np.genfromtxt(fname, dtype=['|S19', 'f8', '|S4'], names=[
                         'path', 'probability', 'type'])
    image_fnames = np.char.decode(data['path'])
    probs = data['probability']
    types = np.char.decode(data['type'])
    
    
    dir = os.path.dirname(fname)

    images = np.array([load_cell_image(os.path.join(dir, fn))
                       for fn in image_fnames])
    
    return images, probs, types

In [4]:
# importing images_names and data from labels.csv file

images_names = [image for image in os.listdir('images')]
images, probs, types = load('labels.csv')

In [5]:
# converting the probabilities into 2 classes
# (0,0.33) = 0
# (0.66 ,1) = 1
'''
for i in range(len(probs)):
    if (probs[i] >= 0) and (probs[i] < 0.34):
        probs[i] = 0
    elif (probs[i] > 0.65) and (probs[i] <= 1):
        probs[i] = 1
'''
#for i in probs:
#    print(i)
print(types)

['mono' 'mono' 'mono' ... 'poly' 'poly' 'poly']


# This Part is only for splitting the data in folders

In [10]:
mono = 0
poly = 0
for name, typ in zip(images_names, types):
    name = 'images/'+ name
    if typ == 'poly':
        poly += 1
        os.rename(name, 'Data/' + f'Poly{poly}' + '.png')
    elif typ == 'mono':
        mono += 1
        os.rename(name, 'Data/' + f'Mono{mono}' + '.png')

In [9]:
src = 'Data'
dsta = 'Data/Mono'
dstb = 'Data/Poly'

for file in os.listdir('Data'):
    if str(file) == 'Mono' or str(file) == 'Poly':
        continue
    src_path = os.path.join(src, file)
    if str(file)[:4] == 'Mono':
        dst_path = os.path.join(dsta, file)
        os.rename(src_path, dst_path)
    elif str(file)[:4] == 'Poly':
        dst_path = os.path.join(dstb, file)
        os.rename(src_path, dst_path)

In [11]:
import splitfolders
splitfolders.ratio('Data', output="ClassifiedByTypes", seed=1337, ratio=(.8, 0.1,0.1)) 

Copying files: 2624 files [00:03, 845.69 files/s]
